In [258]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [259]:
'''About Dataset
Age: Age of patient
Sex: Gender of patient
BP: Blood pressure of patient
Cholesterol: Cholesterol of patient
Na_to_K: Sodium to Potassium Ratio in Blood
Drug: Drug Type
Content
---The target feature is
Drug type
---The feature sets are:
Age
Sex
Blood Pressure Levels (BP)
Cholesterol Levels
Na to Potassium Ration
'''

'About Dataset\nAge: Age of patient\nSex: Gender of patient\nBP: Blood pressure of patient\nCholesterol: Cholesterol of patient\nNa_to_K: Sodium to Potassium Ratio in Blood\nDrug: Drug Type\nContent\n---The target feature is\nDrug type\n---The feature sets are:\nAge\nSex\nBlood Pressure Levels (BP)\nCholesterol Levels\nNa to Potassium Ration\n'

In [260]:
drug = pd.read_csv('drug200.csv')
drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [261]:
print(drug.nunique())
print(drug.shape)

Age             57
Sex              2
BP               3
Cholesterol      2
Na_to_K        198
Drug             5
dtype: int64
(200, 6)


In [262]:
#drug = pd.get_dummies(data=drug, columns=['Sex'])
#drug

In [263]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

In [264]:
#encoding nominal var drug.Sex
# 0 = F, 1 = M
label_enc = LabelEncoder()
label_enc.fit(['F', 'M'])
print(label_enc.classes_)
drug.Sex = label_enc.transform(drug.Sex)
drug.Sex

['F' 'M']


0      0
1      1
2      1
3      0
4      0
      ..
195    0
196    1
197    1
198    1
199    0
Name: Sex, Length: 200, dtype: int32

In [265]:
#encoding ordinal var drug.BP, drug.Cholesterol
ordinal = OrdinalEncoder()
ordinal.fit(drug[['BP']])
drug.BP = ordinal.transform(drug[['BP']])

drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0.0,HIGH,25.355,DrugY
1,47,1,1.0,HIGH,13.093,drugC
2,47,1,1.0,HIGH,10.114,drugC
3,28,0,2.0,HIGH,7.798,drugX
4,61,0,1.0,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,0,1.0,HIGH,11.567,drugC
196,16,1,1.0,HIGH,12.006,drugC
197,52,1,2.0,HIGH,9.894,drugX
198,23,1,2.0,NORMAL,14.020,drugX


In [266]:
ordinal.fit(drug[['Cholesterol']])
drug.Cholesterol = ordinal.transform(drug[['Cholesterol']])
drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0.0,0.0,25.355,DrugY
1,47,1,1.0,0.0,13.093,drugC
2,47,1,1.0,0.0,10.114,drugC
3,28,0,2.0,0.0,7.798,drugX
4,61,0,1.0,0.0,18.043,DrugY
...,...,...,...,...,...,...
195,56,0,1.0,0.0,11.567,drugC
196,16,1,1.0,0.0,12.006,drugC
197,52,1,2.0,0.0,9.894,drugX
198,23,1,2.0,1.0,14.020,drugX


In [267]:
#encoding nominal var y
#['DrugY' = 0, 'drugA' = 1, 'drugB' = 2, 'drugC' = 3, 'drugX' = 4]

label_enc = LabelEncoder()
label_enc.fit(drug.Drug.drop_duplicates())
print(label_enc.classes_)
drug.Drug = label_enc.transform(drug.Drug)
drug

['DrugY' 'drugA' 'drugB' 'drugC' 'drugX']


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0.0,0.0,25.355,0
1,47,1,1.0,0.0,13.093,3
2,47,1,1.0,0.0,10.114,3
3,28,0,2.0,0.0,7.798,4
4,61,0,1.0,0.0,18.043,0
...,...,...,...,...,...,...
195,56,0,1.0,0.0,11.567,3
196,16,1,1.0,0.0,12.006,3
197,52,1,2.0,0.0,9.894,4
198,23,1,2.0,1.0,14.020,4


In [268]:
drug.Cholesterol.head(20)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     1.0
10    0.0
11    1.0
12    0.0
13    0.0
14    0.0
15    1.0
16    1.0
17    0.0
18    0.0
19    1.0
Name: Cholesterol, dtype: float64

In [269]:
drug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    int32  
 2   BP           200 non-null    float64
 3   Cholesterol  200 non-null    float64
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    int32  
dtypes: float64(3), int32(2), int64(1)
memory usage: 7.9 KB


In [270]:
drug.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

In [271]:
x = drug.drop(columns=['Drug'])
y = drug.Drug

print(x.shape)
print(y.shape)

(200, 5)
(200,)


In [272]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(140, 5)
(60, 5)
(140,)
(60,)


In [273]:
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [274]:
from sklearn.metrics import accuracy_score

In [275]:
y_pred = clf.predict(x_test)
accuracy_score(y_test, y_pred)

0.9666666666666667

In [276]:
params = {'max_depth':[x for x in range(1, 11)]}
grid_val = GridSearchCV(clf, param_grid=params, cv=5)

In [277]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=0)
grid_val.fit(x_train, y_train)
y_pred = grid_val.predict(x_test)

In [278]:
print(grid_val.best_estimator_)
print(grid_val.score(x_test, y_test))

DecisionTreeClassifier(max_depth=4)
0.9666666666666667


In [279]:
from sklearn.metrics import classification_report

In [280]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        27
           1       0.88      1.00      0.93         7
           2       1.00      0.80      0.89         5
           3       1.00      1.00      1.00         5
           4       1.00      0.94      0.97        16

    accuracy                           0.97        60
   macro avg       0.97      0.95      0.95        60
weighted avg       0.97      0.97      0.97        60



In [ ]:
# is there imbalanced class?